# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [1]:
library(dplyr)
require(tidyr)
library(ozmaps)
library(galah)
library(data.tree)
library(sf)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr


Attaching package: ‘galah’


The following object is masked from ‘package:tidyr’:

    unnest


The following object is masked from ‘package:dplyr’:

    desc


The following object is masked from ‘package:stats’:

    filter


Linking to GEOS 3.11.0, GDAL 3.5.3, PROJ 9.1.0; sf_use_s2() is TRUE



In [2]:
here::i_am("RDS-output/Read-table-traits-per-species.ipynb")

here() starts at /Users/z3529065/proyectos/fireveg/fireveg-db-exports



In [3]:
galah_config(email = readLines(here::here("secrets","galah-email.txt")))

### Read data

In [4]:
spp_traits_table <- readRDS(here::here("data", "output-Rdata","Summary-traits-species.rds"))


## Query taxonomy from Atlas of Living Australia

In [5]:
nsw <- st_transform(ozmaps::ozmap_states, 4326) |> filter(NAME == "New South Wales")

In [6]:
ALA_taxonomy_file <- here::here('data','ALA','NSW-plants-according-to-ALA.rds')
if (!file.exists(ALA_taxonomy_file)) {
    if (!dir.exists(here::here('data','ALA')))
        dir.create(here::here('data','ALA'))
    plant_families <- galah_call() |>
        galah_identify("plantae") |>
        galah_geolocate(nsw, type = "bbox")|>
        filter(rank == "family") |>
        atlas_taxonomy()
    saveRDS(file=ALA_taxonomy_file,plant_families)
} else {
    plant_families <- readRDS(ALA_taxonomy_file)
}

In [7]:
print(plant_families, pruneMethod = "dist", limit = 10)

# A tibble: 614 × 4
   name             rank     parent_taxon_concept_id            taxon_concept_id
   <chr>            <chr>    <chr>                              <chr>           
 1 Plantae          kingdom  NA                                 https://id.biod…
 2 Anthocerotophyta phylum   https://id.biodiversity.org.au/ta… https://id.biod…
 3 Anthocerotopsida class    https://id.biodiversity.org.au/no… https://id.biod…
 4 Anthocerotidae   subclass https://id.biodiversity.org.au/no… https://id.biod…
 5 Anthocerotales   order    https://id.biodiversity.org.au/no… https://id.biod…
 6 Anthocerotaceae  family   https://id.biodiversity.org.au/no… https://id.biod…
 7 Dendrocerotidae  subclass https://id.biodiversity.org.au/no… https://id.biod…
 8 Dendrocerotales  order    https://id.biodiversity.org.au/no… https://id.biod…
 9 Dendrocerotaceae family   https://id.biodiversity.org.au/no… https://id.biod…
10 Notothylatidae   subclass https://id.biodiversity.org.au/no… https://id.biod…
# ℹ 604 

In [8]:
fams <- plant_families |> 
    filter(rank == "family") |>
    select(family=name, family_concept_id=taxon_concept_id, order_concept_id=parent_taxon_concept_id)
fams <- fams |>
    left_join(plant_families, by=c("order_concept_id" = "taxon_concept_id"))

In [9]:
table(fams$rank)


     order   subclass   suborder superorder 
       405          2         36          3 

### update table with taxonomy
Add column with the plant order from the taxonomy:

In [10]:
spp_traits_table$rank_order <- fams$name[match(spp_traits_table$family,fams$family)]

In [11]:
spp_traits_table |> 
    filter(is.na(rank_order)) |>
           group_by(family) |>
    summarise(spps = n_distinct(spp)) |>
    arrange(desc(spps)) |>
    head()

family,spps
<chr>,<int>
Fabaceae (Faboideae),923
Fabaceae (Mimosoideae),452
Fabaceae (Caesalpinioideae),140
Malaceae,61
Lomandraceae,47
Anthericaceae,37


In [12]:
fabales <- c("Fabaceae (Faboideae)", "Fabaceae (Mimosoideae)", "Fabaceae (Caesalpinioideae)" )
unplaced <- c("Dilleniaceae", "Flacourtiaceae")
single_family <- c("Boraginaceae")
rosales <- c("Malaceae")
lamiales <- c("Myoporaceae", "Buddlejaceae")
polypodiales <- c("Pteridaceae")
asparagales <- c("Anthericaceae","Phormiaceae","Lomandraceae", "Agavaceae", "Hyacinthaceae")
ranunculales <- c("Fumariaceae")
malvales <- c("Tiliaceae")


In [13]:
spp_traits_table <- spp_traits_table %>% mutate(
    rank_order = case_when(
            family %in% fabales ~ "Fabales",
            family %in% unplaced ~ "unplaced",
            family %in% rosales ~ "Rosales",
            family %in% lamiales ~ "Lamiales",
            family %in% polypodiales ~ "Polypodiales",
            family %in% asparagales ~ "Asparagales",
            family %in% ranunculales ~ "Ranunculales",
            family %in% malvales ~ "Malvales",
            family %in% single_family ~ sprintf("fam. %s",family),
            is.na(rank_order) ~ "unknown",
            TRUE ~ rank_order
        ))

In [14]:
spp_traits_table |> 
    filter(rank_order %in% c(NA,"unknown")) |>
           group_by(family) |>
    summarise(spps = n_distinct(spp)) |>
    arrange(desc(spps)) |>
    head()

family,spps
<chr>,<int>
Unknown Flora,32
Athyriaceae,15
Viscaceae,13
Stackhousiaceae,12
Adoxaceae,10
Magnoliaceae,10


Need to check this again, according to David, there should be around 6000 plant species in NSW, is this list including that many synonyms?

In [15]:
table(spp_traits_table$current)


false  true 
 3627 12530 

In [16]:
spp_traits_table %>%
    #filter(current %in% "true") %>%
    group_by(establishment,taxonrank %in% "Species") %>%
    summarise(total=n(), nspp = n_distinct(spp), ncurrent=n_distinct(current_species), .groups = "drop") 

establishment,"taxonrank %in% ""Species""",total,nspp,ncurrent
<chr>,<lgl>,<int>,<int>,<int>
"Alive in NSW, Native",FALSE,1813,1813,1251
"Alive in NSW, Native",TRUE,10556,10556,8169
"Extinct in NSW, Native",FALSE,9,9,4
"Extinct in NSW, Native",TRUE,38,38,28
Hybrid,FALSE,1,1,1
Introduced,FALSE,399,399,242
Introduced,TRUE,3309,3309,2804
Not Known from NSW,FALSE,5,5,5
Not Known from NSW,TRUE,27,27,26


Examine records for species that are not current:

In [17]:
head(subset(spp_traits_table, current %in% "false"))

,family,genus,spp,scientific_name,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1,rank_order
,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
7,Poaceae,Anthosachne,1.167971e-320,Elymus scaber,1.133979e-319,Anthosachne scabra,Species,"Alive in NSW, Native",false,4.940656e-324,⋯,0,0,0,0,0.000000e+00,4.940656e-324,0,4.940656e-324,1.482197e-323,Poales
12,Ericaceae,Agiortia,1.170442e-320,Leucopogon pedicillatus,8.518186e-320,Agiortia pedicellata,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Ericales
14,Fabaceae (Faboideae),Abrus,1.171430e-320,Abrus precatorius,6.625420e-320,Abrus precatorius subsp. precatorius,Subspecies,Introduced,false,0.000000e+00,⋯,0,0,0,0,9.881313e-324,0.000000e+00,0,0.000000e+00,4.940656e-324,Fabales
15,Poaceae,Paractaenum,1.171924e-320,Paractaenum novae-hollandiae subsp. reversum,1.732688e-320,Paractaenum novae-hollandiae,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Poales
17,Pteridaceae,Adiantum,1.172912e-320,Adiantum silvaticum var. silvaticum,3.738595e-320,Adiantum silvaticum,Species,"Alive in NSW, Native",false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Polypodiales
28,Plantaginaceae,Linaria,1.178347e-320,Linaria genistifolia,6.265741e-320,Linaria dalmatica,Species,Introduced,false,0.000000e+00,⋯,0,0,0,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0.000000e+00,Lamiales


In [18]:
spp_traits_table %>% slice(1:5)

family,genus,spp,scientific_name,current_spp,current_species,taxonrank,establishment,current,nquadrat,⋯,repr2,surv5,surv6,surv7,disp1,repr3a,repr3,surv4,surv1,rank_order
<chr>,<chr>,<int64>,<chr>,<int64>,<chr>,<chr>,<chr>,<chr>,<int64>,⋯,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<int64>,<chr>
Brassicaceae,Lepidium,1.165007e-320,Lepidium oxytrichum,1.165007e-320,Lepidium oxytrichum,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,2.470328e-323,0,0.000000e+00,0,1.482197e-323,Brassicales
Myrtaceae,Eucalyptus,1.165501e-320,Eucalyptus williamsiana,1.165501e-320,Eucalyptus williamsiana,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,0.000000e+00,0,0.000000e+00,0,2.964394e-323,Myrtales
Myrtaceae,Melaleuca,1.165995e-320,Melaleuca glomerata,1.165995e-320,Melaleuca glomerata,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,9.881313e-324,0,0.000000e+00,0,2.964394e-323,Myrtales
Apiaceae,Actinotus,1.166489e-320,Actinotus helianthi,1.166489e-320,Actinotus helianthi,Species,"Alive in NSW, Native",true,0,⋯,0,9.881313e-324,0,4.940656e-324,1.482197e-323,0,1.482197e-323,0,7.905050e-323,Apiales
Apiaceae,Apium,1.166983e-320,Apium prostratum,1.166983e-320,Apium prostratum,Species,"Alive in NSW, Native",true,0,⋯,0,0.000000e+00,0,0.000000e+00,1.976263e-323,0,0.000000e+00,0,4.940656e-324,Apiales


## Literature vs. field work data
Let's check how many valid species have information from both sources


In [19]:
 spp_traits_table |> 
    filter(
        #current %in% "true", 
           taxonrank %in% "Species",
           establishment %in% "Alive in NSW, Native") |>
    mutate(
     fieldwork_sources =  nquadrat>0,
     literature_sources = (disp1 +
                           germ1 + germ8 + 
                           rect2 + 
                           grow1 + 
                           repr2 + repr3a + repr3 + repr4 +  
                           surv1 + surv4 + surv5 + surv6 + surv7) > 0
    ) |> 
    group_by(fieldwork_sources, literature_sources) |>
    summarise(total = n_distinct(scientific_name), total_current = n_distinct(current_species), .groups = "drop")

fieldwork_sources,literature_sources,total,total_current
<lgl>,<lgl>,<int>,<int>
FALSE,FALSE,4574,3844
FALSE,TRUE,5129,4843
TRUE,FALSE,87,87
TRUE,TRUE,724,723


In [20]:
4843+87+723

[1] 5653

In [21]:
sum(spp_traits_table$repr2>0)

[1] 218

In [22]:
saveRDS(file=here::here('data', 'output-Rdata', 'Summary-traits-species-orders.rds'),spp_traits_table)